In [ ]:
import shlex
import subprocess

#"eosio-cpp -I/Users/newworld/dev/eosio.contracts/contracts/eosio.system/include -I/Users/newworld/dev/eosio.contracts/contracts/eosio.system/../eosio.token/include -O3 -abigen -abigen_output=/Users/newworld/dev/eosio.contracts/build/contracts/eosio.system/eosio.system.abi -contract eosio.system -o CMakeFiles/eosio.system.dir/src/eosio.system.cpp.obj -c /Users/newworld/dev/eosio.contracts/contracts/eosio.system/src/eosio.system.cpp"

cmds = "eosio-cpp  -O3 -abigen -abigen_output=test.abi -contract test -o test.obj -c test.cpp"
cmds = shlex.split(cmds)
print(cmds)
try:
    ret = subprocess.check_output(cmds, stderr=subprocess.STDOUT)
    print(ret.decode('utf8'))
except Exception as e:
    print(e)

cmds = "eosio-ld  test.obj -o test.wasm"
cmds = shlex.split(cmds)
ret = subprocess.check_output(cmds, stderr=subprocess.STDOUT)
print(ret.decode('utf8'))

In [112]:
import os
import shutil
import hashlib
import marshal
import subprocess
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
wallet.import_key('test', '5JMXaLz5xnVvwrnvAGaZKQZFCDdeU6wjmuJY1rDnXiUZz7Gyi1o')

def find_include_path():
    eosio_link = shutil.which('eosio-cpp')
    eosio_cpp = os.readlink(eosio_link)

    if eosio_cpp[:2] == '..':
        eosio_cpp = os.path.join(os.path.dirname(eosio_link), eosio_cpp)
    eosio_cpp = os.path.abspath(eosio_cpp)
    eosio_cpp = os.path.dirname(eosio_cpp)
    eosio_cpp = os.path.dirname(eosio_cpp)
    return os.path.join(eosio_cpp, 'opt/eosio.cdt/include/eosiolib/capi')


def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)

def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})
print(find_include_path())

True

True

/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/capi


In [30]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [318]:
from pyeoskit import db
db.reset()

code = '''
#include <stdint.h>
#include <eosio/db.h>
#include <eosio/action.h>
#include <eosio/print.h>
#include <string>
using namespace std;
namespace eosio {
    string read_action_data() {
        int size = ::read_action_data(nullptr, 0);
        string s(size, 0);
        ::read_action_data((char *)s.c_str(), s.size());
        return s;
    }
}
extern "C" {
    void apply(uint64_t receiver, uint64_t code, uint64_t action) {
        string s = eosio::read_action_data();
        prints(s.c_str());
        return;
    }
    void call(uint64_t func, uint64_t arg1, uint64_t arg2, uint64_t arg3) {
        string s = eosio::read_action_data();
        prints(s.c_str());
        return;
    }
}
'''

code = r'''
#include <eosio/types.h>
#include <eosio/action.h>
#include <eosio/print.h>
#include <eosio/name.hpp>
extern "C" {

__attribute__((eosio_wasm_import))
int call_contract_get_extra_args(void* extra_args, size_t size1);

__attribute__((eosio_wasm_import))
int call_contract_set_results(void* result, size_t size1);

    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        uint64_t n = 0;
        float f = 10.8;
        for (int i=0;i<100;i++) {
            n += i;
            f += (float)i;
        }
        prints("goodbye,world\n");
    }
    
    __attribute__((eosio_wasm_entry))
    void call(uint64_t func, uint64_t arg1, uint64_t arg2, uint64_t arg3) {
      if (func == eosio::name("calltest1").value) {
         uint64_t extra_args;
         call_contract_get_extra_args(&extra_args, sizeof(extra_args));
         extra_args += 1;
         call_contract_set_results(&extra_args, sizeof(extra_args));
      }
   }
}
'''

r = open('test.cpp', 'w').write(code)

clang_7_args = ['/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/clang-7',
 '-o',
 'test.obj',
 'test.cpp',
 '--target=wasm32',
 '-ffreestanding',
 '-nostdlib',
 '-fno-builtin',
 '-fno-threadsafe-statics',
 '-fno-exceptions',
 '-fno-rtti',
 '-fmodules-ts',
 '-DBOOST_DISABLE_ASSERTS',
 '-DBOOST_EXCEPTION_DISABLE',
 '-Xclang',
 '-load',
 '-Xclang',
 '/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/LLVMEosioApply.dylib',
 '-fplugin=/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/eosio_plugin.dylib',
 '-mllvm',
 '-use-cfl-aa-in-codegen=both',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../include/libcxx',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../include/libc',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../include',
 '--sysroot=/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../include/eosiolib/core',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../include/eosiolib/contracts',
 '-c',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/capi',
 '-I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/core',
 '-O3',
 '--std=c++17',
 ]


# '-e', 'apply',
# '--only-export', 'call:function',
# '--only-export', 'apply:function',

wasm_ld_args = ['/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/wasm-ld',
 '--gc-sections',
 '--strip-all',
 '-zstack-size=8192',
 '--merge-data-segments',
 '-e', 'call',
 '--only-export', 'call:function',
 '-lc++',
 '-lc',
 '-leosio',
 '-leosio_dsm',
 '-mllvm',
 '-use-cfl-aa-in-codegen=both',
 'test.obj',
 '-L/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../lib',
 '-stack-first',
 '--lto-O3',
 '-o',
 'test.wasm',
 '--allow-undefined-file=/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/bin/../eosio.imports']

#%system rm test.obj test.wasm
#%system eosio-cpp -I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/capi -I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/core -O3 -contract test -o test.obj -c test.cpp
#%system eosio-ld test.obj -o test.wasm
#%ls


try:
    ret = subprocess.check_output(clang_7_args)
    print('clang_7 output:', ret.decode('utf8'))
    ret = subprocess.check_output(wasm_ld_args)
    print('wasm_ld output:', ret.decode('utf8'))
except Exception as e:
    print('error', e)


account_name = 'helloworld33'
code = open('test.wasm', 'rb').read()
m = hashlib.sha256()
m.update(code)
code_hash = m.hexdigest()
r = eosapi.get_code(account_name)
if code_hash != r['code_hash']:
    print('update contract')
    abi = ''
    r = eosapi.set_contract(account_name, code, abi, 0)
print('done!')

clang_7 output: 
wasm_ld output: 
done!


In [329]:
from pyeoskit import db
db.reset()
code = '''
def apply(receiver, code, action):
    ret = call_contract('helloworld33', 'calltest1', 0, 0, 0, int.to_bytes(1, 8, 'little'))
    print(ret)
    print(int.from_bytes(ret, 'little'))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

True


279


In [331]:
str = 'hello'
%who 


IPython	 InteractiveShell	 abi	 account_name	 clang_7_args	 cmds	 code	 code_hash	 config	 
eosio_cpp	 eosio_link	 find_include_path	 hashlib	 include_path	 m	 marshal	 name	 os	 
plt	 psw	 publish_contract	 r	 randint	 ret	 run_test_code	 set_abi	 set_code	 
shlex	 shutil	 source	 str	 subprocess	 wallet	 wasm_ld_args	 x	 y	 

